In [26]:
%pip install -U langchain-community

import openai
import langchain
import pinecone
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI


Note: you may need to restart the kernel to use updated packages.


In [27]:
from dotenv import load_dotenv

load_dotenv()



True

In [28]:
import os

In [29]:
# Read the document 
def read_document(file_path):
    file_loader = PyPDFDirectoryLoader(file_path)
    file_loaders = file_loader.load()
    return file_loaders

In [30]:
doc=read_document('documents/')
len(doc)

58

In [31]:
# Divide the document into smaller chunks

def chunk_document(doc):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
    docs = text_splitter.split_documents(doc)
    return docs

In [32]:
documents=chunk_document(doc)
len(documents)

58

In [36]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}  # or "cuda"
)

d:\Projects\E2E_LLM_APP\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\AI_Cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP d

In [37]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [41]:
vectors = embeddings.embed_query("How are you?")
len(vectors)


384

In [44]:
from pinecone import Pinecone, ServerlessSpec

# 1. create the client
pc = Pinecone(
    api_key="pcsk_2FkBj9_FUWpNWG3VEA5M9udA8goVuBV8hWmUdaipdZShFWBjtVAywdshJsDs3125Yr1VXB",
)


In [47]:
texts = ["Cats are great pets.", "Pine trees stay green all year."]
ids   = ["doc1", "doc2"]


In [48]:
#  2. (one-time) create the index if it doesn’t exist
index_name = "langchain-vector"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,               # or whatever your embedding size is
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-west-2"
        )
    )

# 3. connect to the index
index = pc.Index(index_name)

# 3. upsert
index.upsert(vectors=list(zip(ids, embeddings)))

PineconeApiTypeError: Invalid type for variable '0'. Required value type is float and passed type was str at ['values'][0]